# Setup

In [ ]:
# Dependencies

import pandas as pd
import warnings

In [ ]:
# Setting to reveal up to 500 rows in notebook

pd.set_option('display.max_rows', 500)

In [ ]:
# Silence pink warnings

warnings.filterwarnings('ignore')

# Functions

In [ ]:
def basic_clean(a_df):
    '''Performs initial cleaning of dataframe'''
    
    a_df = a_df.drop_duplicates(keep="first")
    
    return a_df

In [ ]:
def all_caps(a_df, a_list):
    '''Renders string df columns in supplied list in all caps, returns dataframe'''
    
    for col in a_list:
        a_df[col] = a_df[col].str.upper()
        
    return a_df

In [ ]:
def vc(a_ser):
    '''Return value_counts().to_frame() for a series'''
    
    return a_ser.value_counts().to_frame()

In [ ]:
def clean_isbns(a_ser):
    '''Renders isbn series data as str and strips unwanted chars, returns a series'''
    
    a_ser = a_ser.astype("str") 
    a_ser = a_ser.str.rstrip(".0")
    
    return a_ser    

In [ ]:
def clean_prices(a_ser):
    '''Strips unwanted chars from price data and renders as float, returns a series'''
    
    a_ser = a_ser.astype("str") 
    a_ser = a_ser.str.lstrip("$")
    a_ser = a_ser.str.replace(",", "", regex=False)
    a_ser = a_ser.str.replace("PRICE NOT YET AVAILABLE**", "0.01", regex=False)
    a_ser = a_ser.str.replace("PREPAID", "0.01", regex=False)
    a_ser = a_ser.apply(lambda x: float(x))

    return a_ser

# Import courses_#.csv files into dataframe, clean and transform data

### *courses_df*

In [ ]:
# Read CSV files from List

csvs_files = ["Resources\\courses_1.csv", "Resources\\courses_2.csv", "Resources\\courses_3.csv"]

course_csv_list = (pd.read_csv(file) for file in csvs_files)

df1 = pd.concat(course_csv_list, ignore_index=True)

In [ ]:
df1

In [ ]:
# Checking shape and dtypes of data

df1.info()

In [ ]:
# Eliminate dupes and drop unneeded columns

df1 = df1.drop_duplicates()

df1 = df1.drop(columns=["Unnamed: 0", "store_id", "catalog_id", "campus", "campus_id", "term_id", "scanDate"])

In [ ]:
# Reorder columns and capitalize data where appropriate

df1 = df1[["department_id", "course_id", "section_id", "university", "term", "department", "course", "section"]]

all_caps(df1, ["university", "term", "department", "course", "section"])

In [ ]:
# Examine value counts on term column to begin consolidation into categories 

vc(df1["term"])

In [ ]:
# Use boolean mask to consolidate Spring 2022 term data 

spring_mask = df1["term"].str.contains("SPRING")

df1.loc[spring_mask, "term"] = "SPRING"


fall_mask = df1["term"].str.contains("FALL")

df1.loc[fall_mask, "term"] = "FALL"


summer_mask = df1["term"].str.contains("SUMMER")

df1.loc[summer_mask, "term"] = "SUMMER"


winter_mask = df1["term"].str.contains("WINTER")

df1.loc[winter_mask, "term"] = "WINTER"



junk_mask = ~(df1["term"].isin(["SPRING", "FALL", "SUMMER", "WINTER"]))

df1.loc[junk_mask, "term"] = "JUNK"

df1 = df1.loc[~(df1["term"]=="JUNK"), :]


vc(df1["term"])

In [ ]:
df1 = df1.reset_index(drop=True)

In [ ]:
courses_df = df1.copy()

In [ ]:
courses_df

# Import textbooks_#.csv files into dataframe, clean and transform data

### *textbooks_df*

In [ ]:
# Read CSV files from List

csvs_files = ["Resources\\textbooks_1.csv", "Resources\\textbooks_2.csv", "Resources\\textbooks_3.csv",\
              "Resources\\textbooks_4.csv", "Resources\\textbooks_5.csv"]

course_csv_list = (pd.read_csv(file) for file in csvs_files)

df2 = pd.concat(course_csv_list, ignore_index=True)

In [ ]:
df2

In [ ]:
# Checking shape and dtypes of data

df2.info()

In [ ]:
# Eliminate dupes and drop unneeded columns

df2 = df2.drop_duplicates()

df2 = df2.drop(columns=\
    ["Unnamed: 0", "store_id", "catalog_id", "campus_id", "term_id", "book_id",\
     "no_textbook_message", "recommend_type", "scanDate"])

In [ ]:
# Reorder columns and capitalize data where appropriate

all_caps(df2, ["title", "edition", "publisher", "book_type", "price"])

In [ ]:
# Drop any rows where title, ISBN, and price are ALL missing (i.e., not much use for analysis)

df2 = df2.dropna(axis=0, subset=["title", "ISBN", "price"], how="all")

In [ ]:
# Fill in missing values with default values  

df2 = df2.fillna(value={"edition":"unknown", "publisher":"unknown", "ISBN": 0.0, "price":"0.01"})

In [ ]:
# Rename ISBN column to isbn 

df2 = df2.rename(columns={"ISBN":"isbn"})

In [ ]:
# Transform isbn data

df2["isbn"] = clean_isbns(df2["isbn"])

In [ ]:
# Transform price data

df2["price"] = clean_prices(df2["price"])

In [ ]:
# Take a random sample of our dataframe to see if transformations are effective

df2.sample(50)

In [ ]:
df2 = df2.reset_index(drop=True)

In [ ]:
textbooks_df = df2.copy()

In [ ]:
textbooks_df.head(5)

In [ ]:
textbooks_df.sample(50)

# Merge courses and textbooks dataframes with inner join

### *merge_courses_and_textbooks_df*

In [ ]:
# Inner merge on both dataframes on common id fields

merge_df = pd.merge(courses_df, textbooks_df, on=["department_id", "course_id", "section_id"], how="inner")

In [ ]:
merge_df.info()

In [ ]:
merge_df.drop_duplicates()

In [ ]:
merge_df = merge_df[["university", "term", "department", "course", "section", "title", "edition", "price",\
                     "isbn", "publisher", "book_type", "book_url"]]

In [ ]:
merge_df.sample(50)

In [ ]:
merge_df = merge_df.reset_index(drop=True)

In [ ]:
merge_df.info()

In [ ]:
course_textbooks = merge_df.copy()

In [ ]:
# Render as csv file for import into PostGresSQL

course_textbooks.to_csv("Resources\\course_textbooks.csv", encoding="utf-8")